In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
%matplotlib inline
import pickle
import pandas as pd
from tasks import wsd
from pathlib import Path
from tasks import wsd
from utils import nlp_tools
from tqdm.auto import tqdm
import numpy as np
import json
from sklearn.metrics import classification_report
from flair.embeddings import TransformerWordEmbeddings
from utils.dataset_download import harvest_data_from_extended_senses
from utils.classificaton_utils import binarize, vectorize_target_expressions,cosine_similiarity,eval_lemma
from torch.optim.adam import Adam
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus
from flair.datasets import TREC_6
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

In [70]:
lemma = 'machine'
pos = 'NN'
senses = {'machine_nn01-38474140'} # machine_nn01-38475772 machine_nn01-38475923 machine_nn01-38475835 machine_nn01-38474140
relations = ['seed','synonym'] # ,'descendant','sibling'
eval_mode = "lemma_etal" # lemma or lemma_etal
    

In [71]:
df_train, df_val, df_test = binarize(lemma,
                        pos,
                        senses, 
                        relations,
                        strict_filter=True,
                        start=1700,
                        end=2000,
                        eval_mode=eval_mode)

# senses before filtering by date = 517
# senses after filtering by date = 433


# of seed senses 26 
# of synonyms 383 
# of branch senses 0


# of seeds selected 1 
# of synonyms selected 44 
# of branches selected 0
[LOG] #rows before removing None vector (1947, 21)
[LOG] #rows after removing None vector (1911, 21)


In [72]:
df_train[df_train.sense_id=='machine_nn01-38474140']

,sense_id,lemma_definition,definition,word_id,lemma,quotation_id,source,text,year,full_text,...,keyword_offset,"vector_bert_base_-1,-2,-3,-4_mean","vector_blert_-1,-2,-3,-4_mean",label,id,daterange,provenance,provenance_type,relation_to_core_senses,relation_to_seed_senses
238,machine_nn01-38474140,"A complex device, consisting of a number of in...","A living body, esp. the human body considered ...",machine_nn01,machine,machine_nn01-38474169,"{'title': 'Death's Vision', 'author': 'J. Reyn...","{'keyword': 'Machins', 'full_text': 'What Nobl...",1709.0,What Nobler Souls the Nobler Machins Wear.,...,29.0,"[0.5628562, -0.04788875, 0.074935675, -0.22630...","[-0.15516208, 0.289941, -0.15124893, -0.206332...",1,machine_nn01-38474140,"{'end': None, 'start': 1604, 'obsolete': False...","[[machine_nn01-38474140, seed, machine_nn01]]",seed,{machine_nn01-38474140},{machine_nn01-38474140}
305,machine_nn01-38474140,"A complex device, consisting of a number of in...","A living body, esp. the human body considered ...",machine_nn01,machine,machine_nn01-38474177,"{'title': 'Spectator', 'author': 'J. Addison',...","{'keyword': 'Machine', 'full_text': 'Cheerfuln...",1712.0,Cheerfulness is..the best Promoter of Health. ...,...,70.0,"[0.0052292813, 0.12355395, 0.023108626, 0.2251...","[-0.04755735, 0.20182909, 0.33001357, -0.04851...",1,machine_nn01-38474140,"{'end': None, 'start': 1604, 'obsolete': False...","[[machine_nn01-38474140, seed, machine_nn01]]",seed,{machine_nn01-38474140},{machine_nn01-38474140}
713,machine_nn01-38474140,"A complex device, consisting of a number of in...","A living body, esp. the human body considered ...",machine_nn01,machine,machine_nn01-38474195,"{'title': 'Med. & Physical Jrnl.', 'author': N...","{'keyword': 'machine', 'full_text': 'When a pr...",1805.0,When a product of diseased action has been eff...,...,82.0,"[0.25928053, 0.049638785, 0.022315167, 0.34901...","[-0.16033216, -0.16846322, 0.5062964, 0.102019...",1,machine_nn01-38474140,"{'end': None, 'start': 1604, 'obsolete': False...","[[machine_nn01-38474140, seed, machine_nn01]]",seed,{machine_nn01-38474140},{machine_nn01-38474140}
938,machine_nn01-38474140,"A complex device, consisting of a number of in...","A living body, esp. the human body considered ...",machine_nn01,machine,machine_nn01-38474223,"{'title': 'Of Human Bondage', 'author': 'W. S....","{'keyword': 'machine', 'full_text': 'He wonder...",1915.0,"He wondered whether at the very end, now that ...",...,50.0,"[0.38040048, 0.38440758, 0.45397452, 0.1211486...","[-0.059219074, 0.23112743, 0.42189148, 0.02944...",1,machine_nn01-38474140,"{'end': None, 'start': 1604, 'obsolete': False...","[[machine_nn01-38474140, seed, machine_nn01]]",seed,{machine_nn01-38474140},{machine_nn01-38474140}
1042,machine_nn01-38474140,"A complex device, consisting of a number of in...","A living body, esp. the human body considered ...",machine_nn01,machine,machine_nn01-38474203,"{'title': 'Poems', 'author': 'W. Wordsworth', ...","{'keyword': 'machine', 'full_text': 'And now I...",1807.0,And now I see with eye serene The very pulse o...,...,52.0,"[-0.46428305, 0.013232344, -0.595714, 0.049642...","[0.021248298, 0.28699854, 0.24638082, -0.01793...",1,machine_nn01-38474140,"{'end': None, 'start': 1604, 'obsolete': False...","[[machine_nn01-38474140, seed, machine_nn01]]",seed,{machine_nn01-38474140},{machine_nn01-38474140}
1056,machine_nn01-38474140,"A complex device, consisting of a number of in...","A living body, esp. the human body considered ...",machine_nn01,machine,machine_nn01-38474212,"{'title': 'Telegraphy', 'author': 'W. H. Preec...","{'keyword': 'machine', 'full_text': 'The human...",1876.0,"The human machine tires, and as a consequence ...",...,10.0,"[0.6930934, 0.09074756, -0.13974331, 0.1105655...","[0.11798739, -0.0029160888, 0.29418808, -0.076...",1,machine_nn01-38474140,"{'end': None, 'start': 1604, 'obsolete': False...","[[machine_nn01-38474140, seed, machine_nn01]]",seed,{machine_nn01-38474140},{machine

In [73]:
df_train.word_id.unique()

array(['body_nn01', 'man_nn01', 'machine_nn01', 'carcass_nn01',
       'person_nn01', 'case_nn02', 'personage_nn01', 'corporeity_nn01',
       'structure_nn01', 'dust_nn01', 'case_nn01', 'automaton_nn01',
       'earth_nn01', 'soma_nn02', 'bulk_nn01', 'microcosm_nn01',
       'personality_nn01', 'tabernacle_nn01', 'vessel_nn01',
       'corpse_nn01', 'case_nn04', 'clay_nn01', 'clod_nn01',
       'skinful_nn01', 'carrion_nn01', 'embodiment_nn01', 'corpus_nn01',
       'flesh_nn01', 'soma_nn01', 'bloodbulk_nn01', 'earth_nn02',
       'soulcase_nn02', 'corporation_nn01', 'chassis_nn01', 'bulk_nn03',
       'bouk_nn01', 'outwall_nn01', 'case_nn03', 'incarnation_nn01',
       'bonehouse_nn01', 'man_nn04', 'bulk_nn02', 'soulcase_nn01',
       'godsimage_nn01', 'quarrons_nn01'], dtype=object)

In [74]:
df_train[df_train.label=="0"]

,sense_id,lemma_definition,definition,word_id,lemma,quotation_id,source,text,year,full_text,...,keyword_offset,"vector_bert_base_-1,-2,-3,-4_mean","vector_blert_-1,-2,-3,-4_mean",label,id,daterange,provenance,provenance_type,relation_to_core_senses,relation_to_seed_senses
0,body_nn01-17170653,The complete physical form of a person or anim...,Particular technical uses. The part of a vehic...,body_nn01,body,body_nn01-132916428,"{'title': 'Material Handling Engin.', 'author'...","{'keyword': 'bodies', 'full_text': 'After car ...",1990.0,"After car bodies are painted, they are moved i...",...,10.0,"[1.2747291, 0.25178745, 0.69486666, 0.42832682...","[1.5054287, 1.1386966, 1.3405375, 0.8012274, -...",0,NaN,NaN,NaN,NaN,NaN,NaN
1,man_nn01-110482153,An adult male human being. Without explicit co...,"As vocative or as int., introducing a remark o...",man_nn01,man,man_nn01-110482440,"{'title': 'Shaela', 'author': 'R. Bulter', 'ge...","{'keyword': 'Min', 'full_text': 'Min A'm vexed...",1976.0,Min A'm vexed ta hear yun.,...,0.0,"[-0.10557328, 0.24347349, 0.731555, -0.4305202...","[-0.49209523, 0.7658461, 0.07512934, 0.0148925...",0,NaN,NaN,NaN,NaN,NaN,NaN
2,body_nn01-17169813,The complete physical form of a person or anim...,Contrasted with the soul. Cf. soul body n. at ...,body_nn01,body,body_nn01-17169857,"{'title': 'Ess. Man', 'author': 'A. Pope', 'ge...","{'keyword': 'Body', 'full_text': 'All are but ...",1733.0,"All are but parts of one stupendous Whole, Who...",...,49.0,"[0.8197431, 0.04237363, 0.6312159, -0.2658673,...","[0.60478234, 0.58020014, 0.053836707, -0.06571...",0,NaN,NaN,NaN,NaN,NaN,NaN
3,machine_nn01-38474877,"A complex device, consisting of a number of in...",A bicycle or tricycle; a motorcycle. Formerly ...,machine_nn01,machine,machine_nn01-38474966,"{'title': 'National Trust Mag.', 'author': Non...","{'keyword': 'machines', 'full_text': 'The cycl...",1992.0,The cyclists..took on the circular 21- or 42-m...,...,92.0,"[-0.18150243, -0.24230756, -0.3336587, 0.34879...","[-0.14852196, 0.69629294, 0.30973893, 0.598406...",0,machine_nn01-38474877,"{'end': None, 'start': 1823, 'obsolete': False...","[[machine_nn01-38474877, seed, machine_nn01]]",seed,{machine_nn01-38474877},{machine_nn01-38474877}
4,carcass_nn01-10177258,The dead body of a person or animal; but no lo...,The naked framework or ‘shell’ of a building b...,carcass_nn01,carcass,carcass_nn01-10177295,"{'title': 'New Pract. Builder', 'author': 'P. ...","{'keyword': 'Carcase', 'full_text': 'Carcase o...",1823.0,"Carcase of a Building, the naked walls, and th...",...,0.0,"[0.6567496, -0.050804906, 0.31024605, 0.059706...","[0.41240987, 0.10217035, 0.48574266, 0.8627304...",0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,man_nn01-110479060,An adult male human being. Without explicit co...,A husband. Now chiefly English regional (north...,man_nn01,man,man_nn01-110479206,"{'title': 'Four Years S. Afr.', 'author': 'C. ...","{'keyword': 'man', 'full_text': 'The wife brok...",1829.0,"The wife broke out, ‘You lament a brother, and...",...,79.0,"[-0.07307064, -0.31692728, 0.38834277, -0.2980...","[-0.20098017, 0.47577783, 0.013388823, -0.2808...",0,NaN,NaN,NaN,NaN,NaN,NaN
1216,person_nn01-30950985,"An individual human being; a man, woman, or ch...",Law. An individual (natural person n.) or corp...,person_nn01,person,person_nn01-30951076,"{'title': 'Daily News', 'author': None, 'gende...","{'keyword': 'persons', 'full_text': 'A Bill..e...",1900.0,"A Bill..extending to juridical persons, that i...",...,31.0,"[0.030711764, 0.28706473, 0.6596842, -0.132111...","[-0.42745396, 0.4621299, 0.34301567, 0.2193956...",0,NaN,NaN,NaN,NaN,NaN,NaN
1217,clay_nn01-9320873,"A stiff viscous earth found, in many varieties...",Short for clay-pipe n. at compounds 2 (colloq...,clay_nn01,clay,clay_nn01-9320896,"{'title': 'Held in Bondage', 'author': '‘Ouida...","{'keyword': 'clays', 'full_text': 'Filthy bird...",1863.0,"Filthy b

In [75]:
df_train.head(3)

,sense_id,lemma_definition,definition,word_id,lemma,quotation_id,source,text,year,full_text,...,keyword_offset,"vector_bert_base_-1,-2,-3,-4_mean","vector_blert_-1,-2,-3,-4_mean",label,id,daterange,provenance,provenance_type,relation_to_core_senses,relation_to_seed_senses
0,body_nn01-17170653,The complete physical form of a person or anim...,Particular technical uses. The part of a vehic...,body_nn01,body,body_nn01-132916428,"{'title': 'Material Handling Engin.', 'author'...","{'keyword': 'bodies', 'full_text': 'After car ...",1990.0,"After car bodies are painted, they are moved i...",...,10.0,"[1.2747291, 0.25178745, 0.69486666, 0.42832682...","[1.5054287, 1.1386966, 1.3405375, 0.8012274, -...",0,NaN,NaN,NaN,NaN,NaN,NaN
1,man_nn01-110482153,An adult male human being. Without explicit co...,"As vocative or as int., introducing a remark o...",man_nn01,man,man_nn01-110482440,"{'title': 'Shaela', 'author': 'R. Bulter', 'ge...","{'keyword': 'Min', 'full_text': 'Min A'm vexed...",1976.0,Min A'm vexed ta hear yun.,...,0.0,"[-0.10557328, 0.24347349, 0.731555, -0.4305202...","[-0.49209523, 0.7658461, 0.07512934, 0.0148925...",0,NaN,NaN,NaN,NaN,NaN,NaN
2,body_nn01-17169813,The complete physical form of a person or anim...,Contrasted with the soul. Cf. soul body n. at ...,body_nn01,body,body_nn01-17169857,"{'title': 'Ess. Man', 'author': 'A. Pope', 'ge...","{'keyword': 'Body', 'full_text': 'All are but ...",1733.0,"All are but parts of one stupendous Whole, Who...",...,49.0,"[0.8197431, 0.04237363, 0.6312159, -0.2658673,...","[0.60478234, 0.58020014, 0.053836707, -0.06571...",0,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
df_train.columns

Index(['sense_id', 'lemma_definition', 'definition', 'word_id', 'lemma',
       'quotation_id', 'source', 'text', 'year', 'full_text', 'keyword',
       'keyword_offset', 'vector_bert_base_-1,-2,-3,-4_mean',
       'vector_blert_-1,-2,-3,-4_mean', 'label', 'id', 'daterange',
       'provenance', 'provenance_type', 'relation_to_core_senses',
       'relation_to_seed_senses'],
      dtype='object')

In [100]:
def enclose_keyword(row,enclose_token='"'):
    """enclose keyword with specific token to point
    learner towards to word it has to focus on
    """
    sentence = ''
    for i,c in enumerate(row.full_text):
        if i == int(row.keyword_offset):
            sentence+=enclose_token + ' '
        elif i ==int(row.keyword_offset + len(row.keyword)):
            sentence+= ' ' + enclose_token
        sentence+=c
    return sentence

#def merge_quotation_gloss(row):
#    out_string = '[GLOSS] '
#    if row.definition:
#        out_string+=row.definition
#    out_string+=' [QUOT] '  
#    if row.enclosed_quotation:
#        out_string+=row.enclosed_quotation
#    return out_string

#def prep_train_text(row):
#    out_string='[TAGET] '+row.keyword+' [TAGET] : '
#    if row.definition:
#        out_string+=row.definition
#    out_string+=' [SEP] '  
#    if row.enclosed_quotation:
#        out_string+=row.enclosed_quotation
#    return out_string

#def prep_test_text(row):
#    out_string='[TAGET] '+row.keyword+' [TAGET] : '
#    if row.enclosed_quotation:
#        out_string+=row.enclosed_quotation
#    return out_string

#def merge_quotation_keyword(row):
#    out_string = '[TARGET] '
#    if row.keyword:
#        out_string+=row.keyword
#    out_string+=' [QUOT] '  
#    if row.enclosed_quotation:
#        out_string+=row.enclosed_quotation
#    return out_string



In [101]:
def to_glossbert_format(df):
    def gloss_string(row, definition):
        out_string=''
        if row.enclosed_quotation:
            out_string+=row.enclosed_quotation
        out_string+=' [SEP] '  
        out_string+=row.keyword+': '
        if row.definition:
            out_string+=definition
        return out_string

    df['enclosed_quotations'] = df.apply(enclose_keyword, axis=1)
    
    rows = [] 
    for i,row in df.iterrows():
        rows.append([gloss_string(row, row.definition), 1])
        definitions = df[df.lemma==row.lemma].definition.unique()
        for d in definitions:
            if d != row.definition:
                rows.append([gloss_string(row,d), 0])
    
    return rows

df_gloss_train = to_glossbert_format(df_train)

In [102]:
df_gloss_train[:10]

[['After car [TARGET] bodies [TARGET] are painted, they are moved into storage to coordinate the production schedule with the number of bodies painted a specific color. [SEP] bodies: Particular technical uses. The part of a vehicle fitted to receive the load; spec. the part of a motor car in which driver and passengers sit, or the fuselage of an aeroplane. Cf. cart-body n. at  cart n. compounds 2, wide-body n.',
  1],
 ['After car [TARGET] bodies [TARGET] are painted, they are moved into storage to coordinate the production schedule with the number of bodies painted a specific color. [SEP] bodies: Contrasted with the soul. Cf. soul body n. at  soul n. compounds 4.',
  0],
 ['After car [TARGET] bodies [TARGET] are painted, they are moved into storage to coordinate the production schedule with the number of bodies painted a specific color. [SEP] bodies: Particular technical uses. The main part of a musical instrument, which in the case of traditional stringed instruments forms a resonati

In [78]:
from pathlib import Path
path = Path('./data/training_data')
path.mkdir(exist_ok=True)

In [79]:
csv_out_path = path / f"{lemma}_{'_'.join(senses)}"
csv_out_path.mkdir(exist_ok=True)


In [80]:
df_train['enclosed_quotation'] = df_train.apply(enclose_keyword, axis=1)
df_train['text'] = df_train.apply(prep_train_text, axis=1)
df_train[['text','label']].to_csv(csv_out_path / "train.csv",index = False, sep='\t')    
df_val['enclosed_quotation'] = df_val.apply(enclose_keyword, axis=1)
df_val['text'] = df_val.apply(prep_test_text, axis=1)
df_val[['text','label']].to_csv(csv_out_path / "dev.csv",index = False, sep='\t')        
df_test['enclosed_quotation'] = df_test.apply(enclose_keyword, axis=1)
df_test['text'] = df_test.apply(prep_test_text, axis=1)
df_test[['text','label']].to_csv(csv_out_path / "test.csv",index = False, sep='\t')    

In [81]:
df_test.text

387     [TAGET] Men [TAGET] : He canton'd out the Coun...
1491    [TAGET] earths [TAGET] : Ley-grounds cannot be...
1841    [TAGET] earth [TAGET] : It is well to see the ...
1244    [TAGET] person [TAGET] : The administrator..ha...
1809    [TAGET] earth [TAGET] : While I drove by in my...
                              ...                        
736     [TAGET] machines [TAGET] : ‘Anyone,’ declared,...
610     [TAGET] machine [TAGET] : To each mortal perad...
1612    [TAGET] body [TAGET] : The coffee, we know, st...
1128    [TAGET] Personalities [TAGET] : Wisdom, Learni...
1281    [TAGET] person [TAGET] : I'm a people [TARGET]...
Name: text, Length: 383, dtype: object

In [82]:
df_train.iloc[0].text

'[TAGET] bodies [TAGET] : Particular technical uses. The part of a vehicle fitted to receive the load; spec. the part of a motor car in which driver and passengers sit, or the fuselage of an aeroplane. Cf. cart-body n. at  cart n. compounds 2, wide-body n. [SEP] After car [TARGET] bodies [TARGET] are painted, they are moved into storage to coordinate the production schedule with the number of bodies painted a specific color.'

In [83]:


# this is the folder in which train, test and dev files reside
data_folder = csv_out_path

# column format indicating which columns hold the text and label(s)
column_name_map = {0: "text", 1: "label"}

# load corpus containing training, test and dev data and if CSV has a header, you can skip it
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=True,
                                         delimiter='\t',    # tab-separated files
) 

2021-01-14 17:14:29,725 Reading data from data/training_data/machine_machine_nn01-38474140
2021-01-14 17:14:29,726 Train: data/training_data/machine_machine_nn01-38474140/train.csv
2021-01-14 17:14:29,727 Dev: data/training_data/machine_machine_nn01-38474140/dev.csv
2021-01-14 17:14:29,727 Test: data/training_data/machine_machine_nn01-38474140/test.csv


In [84]:
# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()
print(label_dict)

2021-01-14 17:14:29,745 Computing label dictionary. Progress:
100%|██████████| 1604/1604 [00:01<00:00, 1060.11it/s]2021-01-14 17:14:31,699 [b'0', b'1']
Dictionary with 2 tags: 0, 1



In [85]:

# 3. initialize transformer document embeddings (many models are available)
document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)

# 4. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, loss_weights={b"1":10, b"0":1}) # loss_weights={"1":10, "0":1}

# 5. initialize the text classifier trainer with Adam optimizer
trainer = ModelTrainer(classifier, corpus, optimizer=Adam)

# 6. start the training
trainer.train('models/taggers/trec',
              learning_rate=1e-5, # use very small learning rate
              mini_batch_size=16,
              mini_batch_chunk_size=4, # optionally set this if transformer is too much for your machine
              max_epochs=10, # terminate after 5 epochs
              )

eps=1e-12, elementwise_affine=True)
            (ffn): FFN(
              (dropout): Dropout(p=0.1, inplace=False)
              (lin1): Linear(in_features=768, out_features=3072, bias=True)
              (lin2): Linear(in_features=3072, out_features=768, bias=True)
            )
            (output_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          )
          (4): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): FFN(
              (dropout): Dropout(p=0.1, inplace=False)
 

{'test_score': 0.9295,
 'dev_score_history': [0.9281,
  0.9248,
  0.9216,
  0.9216,
  0.9248,
  0.9248,
  0.9248,
  0.9248,
  0.9248,
  0.9248],
 'train_loss_history': [0.20128365170646023,
  0.08118115926717782,
  0.019440397426679537,
  0.0010100115429271352,
  0.00028631362048062414,
  0.00015017583772733613,
  0.0002078855192506468,
  0.00012047414655809278,
  0.00012958204591429078,
  5.446238951249556e-05],
 'dev_loss_history': [0.33609411120414734,
  0.5065702795982361,
  0.49905064702033997,
  0.5981439352035522,
  0.6656511425971985,
  0.6893778443336487,
  0.6999250054359436,
  0.7107362747192383,
  0.7207703590393066,
  0.7250329852104187]}